In [11]:
# Use a pipeline as a high-level helper
from transformers import pipeline, AutoTokenizer
from camel_tools.disambig.bert import BERTUnfactoredDisambiguator
from camel_tools.morphology.database import MorphologyDB
from camel_tools.tokenizers.word import simple_word_tokenize
from typing import List
import pprint as pp
import json

In [2]:
disambiguator = BERTUnfactoredDisambiguator.pretrained(use_gpu=True)

Some weights of the model checkpoint at /root/.camel_tools/data/disambig_bert_unfactored/msa were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
def get_morphological_features(sentences: List[List[str]]) -> List[List[dict]]:
        """
        Gets the morphological features for each word in the sentence
        
        Args:
            sentence (List[str]): input sentence tokenized as a list of words, 
                                  must be tokenized on spaces and punctiuation
        Returns:
            features (List[dict]): a list of dictionaries, each dictionary contains the morphological features
        """
        features = disambiguator.tag_sentences(sentences)
        for i in range(len(features)):
            for j in range(len(features[i])):
                features[i][j] = {k: v for k, v in features[i][j].items() if k not in ['stem', 'stemgloss', 'stemcat']}
        return features

In [31]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
tokenizer.pad_token_id = tokenizer.eos_token_id
pipe = pipeline("text-generation",
                model="meta-llama/Llama-3.2-3B-Instruct",
                tokenizer=tokenizer,
                device_map="cuda",
                trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [33]:
with open("../error_types.json") as f:
    err_definitions = json.load(f)
err_def_str = pp.pformat(err_definitions)

system_prompt = [{"role": "system", "content": "You are an AI assistant that is an expert in the arabic language. Your Goal is to create grammatical erros "
                                               "in any given input. You shold only output the version of the input with grammatical errors without "
                                               "adding anything else or changing the formating. You should follow the user's intructions and "
                                               "successfully induce the user's desired error to the input. "
                                               "Here are the definition of common grammatical errors in arabic:"
                                               f"\n{err_def_str}\n"},]

In [46]:
sentences = ['اسأعطيك تقييم جيد']
tokens = [simple_word_tokenize(sentence) for sentence in sentences]
features = get_morphological_features(tokens)
pp.pprint(features)

[[{'asp': 'na',
   'atbseg': 'NOAN',
   'atbtok': 'NOAN',
   'bw': 'اساعطيك/NOUN_PROP',
   'bwtok': 'NOAN',
   'caphi': '2_s_aa_3_t._y_k',
   'cas': 'u',
   'catib6': 'PROP',
   'd1seg': 'NOAN',
   'd1tok': 'NOAN',
   'd2seg': 'NOAN',
   'd2tok': 'NOAN',
   'd3seg': 'NOAN',
   'd3tok': 'NOAN',
   'diac': 'اساعطيك',
   'enc0': '0',
   'form_gen': '-',
   'form_num': '-',
   'gen': '-',
   'gloss': 'NO_ANALYSIS',
   'lex': 'اساعطيك',
   'lex_logprob': -99.0,
   'mod': 'na',
   'num': '-',
   'pattern': 'backoff',
   'per': 'na',
   'pos': 'noun_prop',
   'pos_lex_logprob': -99.0,
   'pos_logprob': -99.0,
   'prc0': '0',
   'prc1': '0',
   'prc2': '0',
   'prc3': '0',
   'rat': 'i',
   'root': 'O',
   'source': 'backoff',
   'stt': 'i',
   'ud': 'PROPN',
   'vox': 'na'},
  {'asp': 'na',
   'atbseg': 'تَقْيِيمَ',
   'atbtok': 'تَقْيِيمَ',
   'bw': 'تَقْيِيم/NOUN+َ/CASE_DEF_ACC',
   'bwtok': 'تَقْيِيم_+َ',
   'caphi': 't_a_q_y_ii_m_a',
   'cas': 'a',
   'catib6': 'NOM',
   'd1seg': 'تَقْيِي

In [49]:
## MT

prompt = system_prompt + \
    [{"role": "user", "content": f"_XC_ - Write the word رائع in a different case. (choose from nominative/accusative/genitive)"}, {"role": "assistant", "content": f"رائعاً"},
             {"role": "user", "content": f"_XC_ - Write the word جميلًا in a different case. (choose from nominative/accusative/genitive)"}, {"role": "assistant", "content": f"جميل"},
             {"role": "user", "content": f"_XC_ - Write the word واحد in a different case. (choose from nominative/accusative/genitive)"}, {"role": "assistant", "content": f"واحدا"},
             {"role": "user", "content": f"_XC_ - Write the word تكريما in a different case. (choose from nominative/accusative/genitive)"}, {"role": "assistant", "content": f"تكريم"},
     {"role": "user", "content": f"_XC_ - Write the verb {tokens[0][1]} in a different tense. (choose from past/present/future)"}]
pp.pprint(prompt)

[{'content': 'You are an AI assistant that is an expert in the arabic '
             'language. Your Goal is to create grammatical erros in any given '
             'input. You shold only output the version of the input with '
             'grammatical errors without adding anything else or changing the '
             "formating. You should follow the user's intructions and "
             "successfully induce the user's desired error to the input. Here "
             'are the definition of common grammatical errors in arabic:\n'
             "{'MG': {'description': 'Words are Merged',\n"
             "        'example': {'correct': 'ذهبت البارحة', 'incorrect': "
             "'ذهبتالبارحة'}},\n"
             " 'MI': {'description': 'Incorrect Word Inflection',\n"
             "        'example': {'correct': 'عارف', 'incorrect': 'معروف'}},\n"
             " 'MT': {'description': 'Incorrect Verb Tense',\n"
             "        'example': {'correct': 'أفرحتني', 'incorrect': "
           

In [50]:
pipe(prompt, max_new_tokens=64)

[{'generated_text': [{'role': 'system',
    'content': "You are an AI assistant that is an expert in the arabic language. Your Goal is to create grammatical erros in any given input. You shold only output the version of the input with grammatical errors without adding anything else or changing the formating. You should follow the user's intructions and successfully induce the user's desired error to the input. Here are the definition of common grammatical errors in arabic:\n{'MG': {'description': 'Words are Merged',\n        'example': {'correct': 'ذهبت البارحة', 'incorrect': 'ذهبتالبارحة'}},\n 'MI': {'description': 'Incorrect Word Inflection',\n        'example': {'correct': 'عارف', 'incorrect': 'معروف'}},\n 'MT': {'description': 'Incorrect Verb Tense',\n        'example': {'correct': 'أفرحتني', 'incorrect': 'تفرحني'}},\n 'OA': {'description': 'Confusion between Yaa / Alif Maksura',\n        'example': {'correct': 'على', 'incorrect': 'علي'}},\n 'OC': {'description': 'Wrong Order of Le